# CNN Model and Nearest Neighbors


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import os

In [ ]:

# Preparing the dataset
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Added validation split

# Training and validation sets
training_set = train_datagen.flow_from_directory('dataset',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='training')

validation_set = train_datagen.flow_from_directory('dataset',
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   subset='validation')


In [ ]:
# # Defining the CNN model
# cnn= Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(10, activation='softmax')
# ])

# # Compiling the model
# cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Training the model
# history = cnn.fit(training_set, validation_data=validation_set, epochs=15)


In [ ]:
# Load ResNet101 model pre-trained on ImageNet, excluding the fully connected layers (include_top=False)




base_model = models.Sequential([
        # Layer 1 - Convolutional Layer
        layers.Conv2D(96, (7, 7), strides=(2, 2), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        
        # Layer 2 - Convolutional Layer
        layers.Conv2D(256, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        
        # Layer 3 - Convolutional Layer
        layers.Conv2D(384, (3, 3), activation='relu'),
        
        # Layer 4 - Convolutional Layer
        layers.Conv2D(384, (3, 3), activation='relu'),
        
        # Layer 5 - Convolutional Layer
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        
        # Flattening and fully connected layers for classification (you can add any FC layers here)
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')  # Adjust the number of classes if needed
    ])

# Freeze the convolutional layers (feature extraction)
base_model.trainable = False

# Add new fully connected layers for classification
new_classifier = Sequential([
    GlobalAveragePooling2D(),  # Add a GlobalAveragePooling layer to reduce dimensions
    Dense(64, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout for regularization
    Dense(10, activation='softmax')  # Output layer (adjust number of classes)
])

# Combine the ResNet101 base model and the new classifier
cnn_model = Sequential([
    base_model,  # Add ResNet101 as the base
    new_classifier  # Add the new classifier on top
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the new dataset (only the fully connected layers will be trained)
history = cnn_model.fit(training_set, validation_data=validation_set, epochs=15)

In [ ]:
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# import numpy as np

# img = load_img('photos/plan.jpg', target_size=(224, 224))
# img_array = img_to_array(img) / 255.0  # Rescale to [0, 1]
# img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# # Perform prediction
# predictions = model.predict(img_array)
# predicted_class_index = np.argmax(predictions)  # Get the index of the highest probability
# class_labels = training_set.class_indices  # Get mapping of class labels to indices
# class_labels = {v: k for k, v in class_labels.items()}  # Reverse the dictionary

# # Get the class name
# predicted_class = class_labels[predicted_class_index]
# confidence = predictions[0][predicted_class_index]

# print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}")  


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Function to extract features from ResNet101 base (without the new fully connected layers)
def extract_features_fixed(image_path, base_model):
    img = load_img(image_path, target_size=(224, 224))  # Load image
    img_array = img_to_array(img) / 255.0  # Preprocess the image (normalize)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    features = base_model.predict(img_array)  # Extract features from ResNet101 base
    return features.flatten()  # Flatten the features to 1D

# Function to create feature list for nearest neighbors
def create_feature_list(dataset_dir, base_model):
    features_list = []
    image_paths = []
    
    for subdir, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_path = os.path.join(subdir, file)
                features = extract_features_fixed(file_path, base_model)
                features_list.append(features)
                image_paths.append(file_path)
    
    return np.array(features_list), image_paths

# Create feature list for the whole dataset
features_list, image_paths = create_feature_list("dataset", base_model)

In [ ]:
# Function to find the nearest images based on extracted features
def find_nearest_images(image_path, base_model, n_neighbors=10):
    # Extract features for the input image
    image_features = extract_features_fixed(image_path, base_model)
    
    # Using NearestNeighbors to find the nearest images
    neighbors = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    neighbors.fit(features_list)
    
    distances, indices = neighbors.kneighbors([image_features])
    
    # Print the nearest images and their distance
    print(f"Nearest images to {image_path}:")
    for i in range(n_neighbors):
        print(f"{image_paths[indices[0][i]]} - Distance: {distances[0][i]:.4f}")
    
    # Optionally, plot the nearest images
    plt.figure(figsize=(12, 6))
    for i in range(n_neighbors):
        img = load_img(image_paths[indices[0][i]], target_size=(224, 224))
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Distance: {distances[0][i]:.4f}")
    plt.show()

# Example usage: Finding nearest images
find_nearest_images('vezel.jpg', base_model, n_neighbors=10)